<a href="https://colab.research.google.com/github/vandrearczyk/hecktor-euvip2024/blob/main/baseline_classification_hecktor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split, permutation_test_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import ttest_rel

from google.colab import files

In [52]:
def load_features(folder_path, file_start=""):
    """
    Load all CSV files from a specified folder and concatenate them into a single DataFrame.

    Args:
    folder_path (str): Path to the folder containing CSV files.

    Returns:
    pd.DataFrame: Combined DataFrame from all CSV files.
    """
    dfs = []
    for filename in os.listdir(folder_path):
        if filename.startswith(file_start) and filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path)
            dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

def preprocess_data(df):
    """
    Preprocess the DataFrame by keeping the first three columns.
    Then pivot the table to combine 'Modality', 'ROI', and each feature.

    Args:
    df (pd.DataFrame): Combined DataFrame from multiple CSV files.

    Returns:
    pd.DataFrame: Pivoted DataFrame ready for model training.
    """
    # Keep the first three columns
    first_three_columns = df.iloc[:, :3]

    # Keep all columns
    filtered_df = df

    # Melt the filtered DataFrame
    feature_columns = [col for col in filtered_df.columns if col not in first_three_columns.columns]
    melted_df = filtered_df.melt(id_vars=['PatientID', 'Modality', 'ROI'], value_vars=feature_columns, var_name='Feature')

    # Create combined feature names
    melted_df['Combined'] = melted_df['ROI'] + '_' + melted_df['Modality'] + '_' + melted_df['Feature']

    # Pivot the DataFrame
    pivoted_df = melted_df.pivot_table(index='PatientID', columns='Combined', values='value')
    pivoted_df.reset_index(inplace=True)

    print("Number of features: ", pivoted_df.shape[1])

    return pivoted_df

def filter_patients(df1, df2):
    """
    Filter out patients not present in both DataFrames df1 and df2
    based on the 'PatientID' column.

    Args:
    df1 (pd.DataFrame): DataFrame containing patient features.
    df2 (pd.DataFrame): DataFrame containing patient survival data.

    Returns:
    tuple: A tuple containing the filtered pivoted_df and df2 DataFrames.
    """

    # Identify patients to be deleted from each DataFrame
    deleted_from_df2 = set(df2['PatientID']) - set(df1['PatientID'])
    deleted_from_df1 = set(df1['PatientID']) - set(df2['PatientID'])

    # Print details of deleted patients
    print("Deleted patients from df2:", len(deleted_from_df2), deleted_from_df2)
    print("Deleted patients from df1:", len(deleted_from_df1), deleted_from_df1)

    # Filter DataFrames to only include matching patients
    df1 = df1[df1['PatientID'].isin(df2['PatientID'])]
    df2 = df2[df2['PatientID'].isin(df1['PatientID'])]

    # Print the number of remaining patients
    print("Remaining patients:", df1.shape[0])

    return df1, df2

def extract_components(df):
    # Clean column names to remove spaces
    df.columns = df.columns.str.replace(' ', '', regex=False)

    # Initialize sets to store unique components
    rois = set()
    modalities = set()
    originals = set()
    families = set()
    features = set()

    # Iterate over the columns and extract components
    for col in df.columns:
        # Skip 'PatientID' and 'Split' columns
        if col in ['PatientID', 'Split']:
            continue

        # Split the column name into components
        col_parts = col.split('_')

        # Add each component to its corresponding set
        rois.add(col_parts[0])
        modalities.add(col_parts[1])
        originals.add(col_parts[2])
        families.add(col_parts[3])
        features.add(col_parts[4])

    # Convert sets to sorted lists
    return {
        'rois': sorted(rois),
        'modalities': sorted(modalities),
        'originals': sorted(originals),
        'families': sorted(families),
        'features': sorted(features)
    }

def select_feature(df, feature_dict=None):
    # Remove all spaces from column names
    df.columns = df.columns.str.replace(' ', '', regex=False)

    # Define the prefix for columns that should be kept (PatientID and Split)
    keep_columns = ['PatientID', 'Split']

    # If feature_dict is not provided, set it to an empty dictionary
    if feature_dict is None:
        feature_dict = {}

    # Extract filters from the dictionary
    rois = feature_dict.get('rois', None)
    modalities = feature_dict.get('modalities', None)
    originals = feature_dict.get('originals', None)
    families = feature_dict.get('families', None)
    features = feature_dict.get('features', None)

    # Iterate over all columns in the dataframe
    for col in df.columns:
        # Skip the PatientID and Split columns
        if col in keep_columns:
            continue

        # Split the column name into parts
        col_parts = col.split('_')

        # Unpack the parts based on the known pattern
        roi, modality, orig, family, feature = col_parts[0], col_parts[1], col_parts[2], col_parts[3], col_parts[4]

        # Check if this column should be included based on the filters provided
        if ((rois is None or roi in rois) and
            (modalities is None or modality in modalities) and
            (originals is None or orig in originals) and
            (families is None or family in families) and
            (features is None or feature in features)):
            keep_columns.append(col)

    # Return the dataframe with only the selected columns
    return df[keep_columns]

def fill_missing_values(X):
    """
    Fills missing values in a DataFrame:
    - For numerical columns: fills with the column mean.
    - For non-numerical columns: fills with the most frequent value (mode).

    Parameters:
    X (pd.DataFrame): The DataFrame to fill missing values in.

    Returns:
    pd.DataFrame: The DataFrame with missing values filled.
    """

    # Separate the DataFrame into numerical and non-numerical columns
    numerical_cols = X.select_dtypes(include=['number']).columns
    non_numerical_cols = X.select_dtypes(exclude=['number']).columns

    # Fill missing values in numerical columns with the mean
    X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].mean())

    # Fill missing values in non-numerical columns with the mode (most frequent value)
    for col in non_numerical_cols:
        X[col] = X[col].fillna(X[col].mode()[0])

    return X

def scale_features(X):
    """
    Scales numerical features in a DataFrame using StandardScaler and keeps non-numerical features unchanged.

    Parameters:
    X (pd.DataFrame): The DataFrame to scale features in.

    Returns:
    pd.DataFrame: The DataFrame with scaled numerical features and unchanged non-numerical features.
    """

    # Separate the DataFrame into numerical and non-numerical columns
    numerical_cols = X.select_dtypes(include=['number']).columns
    non_numerical_cols = X.select_dtypes(exclude=['number']).columns

    # Initialize the scaler
    scaler = StandardScaler()

    # Scale the numerical columns
    X_scaled_numerical = pd.DataFrame(scaler.fit_transform(X[numerical_cols]),
                                      columns=numerical_cols,
                                      index=X.index)

    # Combine scaled numerical columns with original non-numerical columns
    X_scaled = pd.concat([X_scaled_numerical, X[non_numerical_cols]], axis=1)

    return X_scaled

def bootstrap_analysis(X, y, model, n_bootstrap=1000):
    boot_accuracies = []
    boot_roc_aucs = []

    for _ in range(n_bootstrap):
        # Resample with replacement
        X_resample, y_resample = resample(X, y, random_state=_)

        # Make predictions on the resampled data
        y_pred_resample = model.predict(X_resample)

        # Calculate metrics
        accuracy = accuracy_score(y_resample, y_pred_resample)
        roc_auc = roc_auc_score(y_resample, y_pred_resample)

        boot_accuracies.append(accuracy)
        boot_roc_aucs.append(roc_auc)

    # Calculate the mean and 95% confidence intervals
    accuracy_mean = np.mean(boot_accuracies)
    accuracy_ci_lower = np.percentile(boot_accuracies, 2.5)
    accuracy_ci_upper = np.percentile(boot_accuracies, 97.5)

    roc_auc_mean = np.mean(boot_roc_aucs)
    roc_auc_ci_lower = np.percentile(boot_roc_aucs, 2.5)
    roc_auc_ci_upper = np.percentile(boot_roc_aucs, 97.5)

    return (accuracy_mean, accuracy_ci_lower, accuracy_ci_upper), (roc_auc_mean, roc_auc_ci_lower, roc_auc_ci_upper)

def evaluate_model(model, X_train, y_train, X_test, y_test, run_bootstrap=False):
    # Evaluate the model
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    roc_auc_train = roc_auc_score(y_train, y_train_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred)

    print(f'Accuracy (Train): {train_accuracy:.2f}')
    print(f'Accuracy (Test): {test_accuracy:.2f}')
    print(f'ROC AUC (Train): {roc_auc_train:.2f}')
    print(f'ROC AUC (Test): {roc_auc_test:.2f}')

    # Perform bootstrap analysis if required
    if run_bootstrap:
        accuracy_results, roc_auc_results = bootstrap_analysis(X_test, y_test, model)

        # Output bootstrap results
        print(f"Bootstrap Accuracy (Test): {accuracy_results[0]:.3f} (95% CI: {accuracy_results[1]:.3f} - {accuracy_results[2]:.3f})")
        print(f"Bootstrap ROC AUC (Test): {roc_auc_results[0]:.3f} (95% CI: {roc_auc_results[1]:.3f} - {roc_auc_results[2]:.3f})")

    # Detailed classification report
    print("Classification Report (Test):\n", classification_report(y_test, y_test_pred))

def compare_models_statistically(model1, model2, X_test1, X_test2, y_test):
    # Get predictions
    y_pred1 = model1.predict(X_test1)
    y_pred2 = model2.predict(X_test2)

    # Calculate accuracy for both models
    accuracy1 = accuracy_score(y_test, y_pred1)
    accuracy2 = accuracy_score(y_test, y_pred2)

    # Paired t-test for comparing model performance (accuracy)
    t_stat, p_value_ttest = ttest_rel(y_pred1, y_pred2)

    # McNemar's test for model comparison on the same dataset
    contingency_table = confusion_matrix(y_pred1 == y_test, y_pred2 == y_test)
    mcnemar_result = mcnemar(contingency_table, exact=False)

    # Print the results
    print(f"Accuracy of model1: {accuracy1:.4f}")
    print(f"Accuracy of model2: {accuracy2:.4f}")
    print("\nPaired t-test results:")
    print(f"t-statistic: {t_stat:.4f}")
    print(f"p-value: {p_value_ttest:.4f}")

    print("\nMcNemar's test results:")
    print(f"statistic: {mcnemar_result.statistic:.4f}")
    print(f"p-value: {mcnemar_result.pvalue:.4f}")

    # Interpretation
    if p_value_ttest < 0.05:
        print("\nPaired t-test: Significant difference between the two models.")
    else:
        print("\nPaired t-test: No significant difference between the two models.")

    if mcnemar_result.pvalue < 0.05:
        print("McNemar's test: Significant difference between the two models.")
    else:
        print("McNemar's test: No significant difference between the two models.")


In [3]:
# Upload features
if any(fn.startswith('features_album') for fn in os.listdir('.')):
  print('Features already uploaded')
else:
  uploaded = files.upload()

Saving features_album_HECKTOR_EUVIP2024_HPV_PT- GTVp.csv to features_album_HECKTOR_EUVIP2024_HPV_PT- GTVp.csv
Saving features_album_HECKTOR_EUVIP2024_HPV_PT- GTVn.csv to features_album_HECKTOR_EUVIP2024_HPV_PT- GTVn.csv
Saving features_album_HECKTOR_EUVIP2024_HPV_CT- GTVp.csv to features_album_HECKTOR_EUVIP2024_HPV_CT- GTVp.csv
Saving features_album_HECKTOR_EUVIP2024_HPV_CT- GTVn.csv to features_album_HECKTOR_EUVIP2024_HPV_CT- GTVn.csv


In [7]:
# Upload hpv outcome
if os.path.exists('hpv_outcome.csv'):
  print('Outcome data already uploaded')
else:
  uploaded = files.upload()

Saving hpv_outcome_full.csv to hpv_outcome_full.csv


In [5]:
# Upload patient split
if any(fn.startswith('patient_split') for fn in os.listdir('.')):
  print('Patient split already uploaded')
else:
  uploaded = files.upload()

Saving patient_split.csv to patient_split.csv


In [6]:
# Upload clinical info
if any(fn.startswith('clinical_info') for fn in os.listdir('.')):
  print('Clinical info already uploaded')
else:
  uploaded = files.upload()

Saving clinical_info.csv to clinical_info.csv


In [43]:
# Load the data (features, outcomes and train/test split)
features_df = load_features(folder_path='./', file_start="features_album")
outcome_df = pd.read_csv('hpv_outcome_full.csv')
split_df = pd.read_csv('patient_split.csv')
clinical_df = pd.read_csv('clinical_info.csv')
print(features_df.shape,outcome_df.shape,split_df.shape, clinical_df.shape)

(408, 115) (524, 2) (102, 2) (524, 10)


In [44]:
# Preprocess the data
features_df = preprocess_data(features_df)
# Filter out patients if not present in features or outcome data
features_df, outcome_df = filter_patients(features_df, outcome_df)
clinical_df, outcome_df = filter_patients(clinical_df, outcome_df)
# Merge split_df with features_df and outcome_df to ensure indices are aligned
features_df = features_df.merge(split_df[['PatientID', 'Split']], on='PatientID')
outcome_df = outcome_df.merge(split_df[['PatientID', 'Split']], on='PatientID')
clinical_df = clinical_df.merge(split_df[['PatientID', 'Split']], on='PatientID')

# Print all available features (rois, modalities, families and feature names)
print(extract_components(features_df))

Number of features:  359
Deleted patients from df2: 422 {'MDA-115', 'MDA-047', 'MDA-154', 'HGJ-074', 'CHUM-046', 'MDA-127', 'CHUV-036', 'CHUS-061', 'CHUP-062', 'MDA-167', 'HGJ-052', 'CHUV-026', 'MDA-139', 'MDA-030', 'MDA-055', 'MDA-108', 'CHUM-061', 'MDA-128', 'MDA-112', 'HMR-013', 'HGJ-007', 'HMR-028', 'MDA-183', 'CHUP-065', 'MDA-095', 'CHUM-002', 'MDA-043', 'MDA-094', 'CHUV-022', 'MDA-086', 'MDA-133', 'CHUS-060', 'MDA-153', 'MDA-042', 'MDA-070', 'MDA-031', 'MDA-097', 'HGJ-077', 'MDA-106', 'CHUV-029', 'MDA-049', 'MDA-075', 'MDA-022', 'CHUS-036', 'CHUP-000', 'CHUS-040', 'CHUV-040', 'MDA-037', 'CHUV-008', 'MDA-026', 'HGJ-031', 'CHUP-018', 'MDA-045', 'MDA-072', 'CHUV-050', 'MDA-080', 'HGJ-073', 'CHUM-032', 'MDA-093', 'CHUS-080', 'CHUS-022', 'MDA-029', 'MDA-020', 'MDA-181', 'MDA-110', 'CHUV-033', 'HGJ-030', 'MDA-163', 'MDA-122', 'MDA-088', 'CHUS-050', 'CHUP-002', 'MDA-155', 'CHUM-008', 'MDA-044', 'MDA-078', 'MDA-125', 'MDA-111', 'CHUM-040', 'MDA-007', 'CHUS-100', 'CHUP-073', 'CHUV-035', '

In [54]:
# Select specific set of features
features_dict = {
    'rois':['GTVp','GTVn'],
    'modalities':['CT','PT'],
    'originals':None,
    'families':None,
    'features':None
}
features_df1 = select_feature(features_df, features_dict)
print(features_df1.shape)

# Select clinical infos
clinical_features = [] # 'Age', 'Gender'
clinical_features.insert(0,'PatientID')
clinical_df1 = clinical_df[clinical_features]

# Merge with clinical features
features_df1 = pd.merge(features_df1, clinical_df1, on='PatientID', how='inner')
print(features_df1.shape)

(102, 360)
(102, 360)


In [55]:
# Prepare data for training
X = features_df1.drop(columns=['PatientID', 'Split'])
X = fill_missing_values(X)
print("Number of features:", X.shape[1])
y = outcome_df['Outcome']

# Ensure the index of y matches the merged_df index
y = y.loc[features_df1.index]

# Rescale the features (only if numerical)
X_scaled = scale_features(X)

# Split the dataset into training and testing sets based on the 'Split' column
X_train = X_scaled[features_df1['Split'] == 'train']
X_test = X_scaled[features_df1['Split'] == 'test']
y_train = y[features_df1['Split'] == 'train']
y_test = y[features_df1['Split'] == 'test']

# Feature selection
# selector = SelectKBest(mutual_info_classif, k=20)
# X_train = selector.fit_transform(X_train, y_train)
# X_test = selector.transform(X_test)
# print("Number of features after selection:", X_test.shape[1])

# Define the model
# model = RandomForestClassifier(n_estimators=100, min_samples_split=10, min_samples_leaf=20, random_state=42)
model = RandomForestClassifier(n_estimators=200, min_samples_split=5, min_samples_leaf=10, max_depth=None, random_state=42)
# model = RandomForestClassifier(random_state=42)
# model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
evaluate_model(model, X_train, y_train, X_test, y_test, run_bootstrap=True)

Number of features: 358
Accuracy (Train): 0.83
Accuracy (Test): 0.75
ROC AUC (Train): 0.83
ROC AUC (Test): 0.75
Bootstrap Accuracy (Test): 0.755 (95% CI: 0.550 - 0.950)
Bootstrap ROC AUC (Test): 0.755 (95% CI: 0.559 - 0.938)
Classification Report (Test):
               precision    recall  f1-score   support

         0.0       0.78      0.70      0.74        10
         1.0       0.73      0.80      0.76        10

    accuracy                           0.75        20
   macro avg       0.75      0.75      0.75        20
weighted avg       0.75      0.75      0.75        20



In [ ]:
# Select specific set of features
rois = ['GTVp']
modalities = ['PT']
originals = None
families = ['shape']
features = None
features_df2 = select_feature(features_df, rois, modalities, originals, families, features)
print(features_df2.shape)

(102, 16)


In [ ]:
# Prepare data for training
X2 = features_df2.drop(columns=['PatientID', 'Split'])
X2 = X2.fillna(X2.mean())
print("Number of features:", X2.shape[1])

# Split the dataset into training and testing sets based on the 'Split' column
X_train2 = X2[features_df2['Split'] == 'train']
X_test2 = X2[features_df2['Split'] == 'test']

# Feature selection
# selector = SelectKBest(mutual_info_classif, k=20)
# X_train = selector.fit_transform(X_train, y_train)
# X_test = selector.transform(X_test)
# print("Number of features after selection:", X_test.shape[1])

# Train the model
# model = RandomForestClassifier(n_estimators=100, min_samples_split=10, min_samples_leaf=20, random_state=42)
model2 = RandomForestClassifier(n_estimators=200, min_samples_split=5, min_samples_leaf=10, max_depth=None, random_state=42)
# model = RandomForestClassifier(random_state=42)
# model = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=3, random_state=42)
model2.fit(X_train2, y_train)

# Evaluate the model
evaluate_model(model2, X_train2, y_train, X_test2, y_test, run_bootstrap=True)

Number of features: 14
Accuracy (Train): 0.71
Accuracy (Test): 0.45
ROC AUC (Train): 0.70
ROC AUC (Test): 0.45
Bootstrap Accuracy (Test): 0.442 (95% CI: 0.250 - 0.650)
Bootstrap ROC AUC (Test): 0.444 (95% CI: 0.229 - 0.660)
Classification Report (Test):
               precision    recall  f1-score   support

         0.0       0.43      0.30      0.35        10
         1.0       0.46      0.60      0.52        10

    accuracy                           0.45        20
   macro avg       0.45      0.45      0.44        20
weighted avg       0.45      0.45      0.44        20



In [ ]:
compare_models_statistically(model, model2, X_test, X_test2, y_test)

Accuracy of model1: 0.7500
Accuracy of model2: 0.4500

Paired t-test results:
t-statistic: -0.6227
p-value: 0.5409

McNemar's test results:
statistic: 2.5000
p-value: 0.1138

Paired t-test: No significant difference between the two models.
McNemar's test: No significant difference between the two models.


In [ ]:
# To do:
# Add models: LogisticRegression, SVC, DecisionTree
# statistical test: permutation_test_score
# Add clinical info
# Check center-only perf

(array([1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        1., 0., 1.]),
 array([1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        0., 0., 0.]))